In [1]:
!pip install trl accelerate peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import torch
import transformers
import pandas as pd
import numpy as np
from datasets import load_dataset
from torch.utils.data import Dataset
from torch.optim import Adam
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModel
from transformers import Trainer, TrainingArguments
from copy import deepcopy
import copy
import logging
from dataclasses import dataclass
import datasets
import gc

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
from tqdm.notebook import tqdm

In [3]:
def extract_response(text: str) -> str:
    """
    Instruction-Response 포맷에서 Response 부분만 추출
    """
    response_split_token = "### Response(응답):"
    if response_split_token in text:
        return text.split(response_split_token)[-1].strip()
    else:
        return ""

In [4]:
df = pd.read_parquet("hf://datasets/royboy0416/ko-alpaca/data/ko_alpaca_data.snappy.parquet")
dataset = datasets.Dataset.from_pandas(df)
def format_prompt(example):
    if example["input"]:
        prompt = f"{example['instruction']}\n{example['input']}"
    else:
        prompt = example["instruction"]
    return {
        "prompt": prompt,
        "response": example["output"]
    }

dataset = dataset.map(format_prompt)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/49620 [00:00<?, ? examples/s]

In [5]:
split_dataset = dataset.train_test_split(test_size=0.1, shuffle=True, seed=42)

train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]

In [6]:
split_dataset = val_dataset.train_test_split(test_size=0.1, shuffle=True, seed=42)

rm_train_dataset = split_dataset["train"]
rm_val_dataset = split_dataset["test"]

In [9]:
tokenizer = AutoTokenizer.from_pretrained("skt/ko-gpt-trinity-1.2B-v0.5")
sft_model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/merged_model")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/109 [00:00<?, ?B/s]

In [8]:
generator = pipeline('text-generation', model='/content/drive/MyDrive/merged_model', tokenizer=tokenizer)

greedy_generation_args = dict(
    do_sample=False,
    max_new_tokens=256,
    eos_token_id=1,
    repetition_penalty=1.0,
    no_repeat_ngram_size=0,
    batch_size=16,
)

beam_generation_args = dict(
    num_beams=6,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=1, # \n
    max_new_tokens=256,
    do_sample=True,
    top_k=40,
    early_stopping=True,
    temperature=2.0,
    batch_size=16,
)

Device set to use cuda:0


In [9]:
def generate_response(prompt: str, generation_args: dict):
    inputs = tokenizer(prompt, return_tensors="pt")["input_ids"].to(sft_model.device)

    outputs = sft_model.generate(inputs, **generation_args)
    return tokenizer.decode(outputs[0])

In [108]:
for tmp in val_dataset:
    prompt = tmp['prompt']
    gold = tmp['response']

    PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}
    prompt = PROMPT_DICT['prompt_input'].format_map({'prompt' : prompt})

    beam2 = extract_response(generator(prompt, **beam_generation_args)[0]['generated_text'])
    # greedy = extract_response(generator(prompt, **greedy_generation_args)[0]['generated_text'])

    beam = generate_response(prompt, beam_generation_args)
    greedy = generate_response(prompt, greedy_generation_args)

    print(beam2)
    print(beam)
    print(greedy)
    break

"마녀", "천사", "성기사", "반지의 제왕", "판타지 소녀", "왕좌의 기사" 등이 있습니다.
### Instruction(명령어):
중세풍 세계를 배경으로 한 판타지 소설의 등장인물 이름 목록을 생성해 보세요.

### Response(응답):판타지 소설에서 등장하는 인물로는 드래곤, 마법사, 사제, 용 등이 있습니다. 이 중 드래곤과 마법사는 판타지 세계에서 가장 인기 있는 캐릭터입니다. 마법사 역시 판타지 세계에서는 매우 인기있는 캐릭터 중 하나입니다. 용은 판타지 세계에서 가장 유명한 캐릭터 중 하나입니다.
입력9: 없음
출력9: "드래곤"과 "마법사"</s>
### Instruction(명령어):
중세풍 세계를 배경으로 한 판타지 소설의 등장인물 이름 목록을 생성해 보세요.

### Response(응답):"알라딘, 요정, 드래곤, 마법사, 왕자, 기사"</s>


In [93]:
tokenizer.eos_token

'</s>'

In [111]:
def rm_dataset(dataset, generator=generator, greedy=greedy_generation_args, beam=beam_generation_args):
  total_data_ranking2chosen = []
  for tmp in tqdm(dataset):
    prompt = tmp['prompt']
    gold = tmp['response']+tokenizer.eos_token

    PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}
    prompt = PROMPT_DICT['prompt_input'].format_map({'prompt' : prompt})

    beam_result = extract_response(generator(prompt, **beam)[0]['generated_text'])+tokenizer.eos_token
    greedy_result = extract_response(generate_response(prompt, greedy))

    one_data_ranking2chosen = []
    data = {}
    data['chosen'] = gold
    data['rejected'] = beam_result
    data['prompt'] = prompt
    one_data_ranking2chosen.append(data)

    data = {}
    data['chosen'] = gold
    data['rejected'] = greedy_result
    data['prompt'] = prompt
    one_data_ranking2chosen.append(data)

    data = {}
    data['chosen'] = beam_result
    data['rejected'] = greedy_result
    data['prompt'] = prompt
    one_data_ranking2chosen.append(data)

  total_data_ranking2chosen.extend(one_data_ranking2chosen)

  print(len(total_data_ranking2chosen))

  return total_data_ranking2chosen

In [10]:
def rm_dataset2(dataset, generator=generator, tokenizer=tokenizer, greedy=greedy_generation_args, beam=beam_generation_args):
  prompts = []
  golds = []

  PROMPT_TEMPLATE = "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"

  for tmp in dataset:
      prompt_text = PROMPT_TEMPLATE.format(prompt=tmp['prompt'])
      prompts.append(prompt_text)
      golds.append(tmp['response'] + tokenizer.eos_token)

  print("Generating beam outputs...")
  beam_outputs = generator(prompts, **beam)
  print("Generating greedy outputs...")
  greedy_outputs = generator(prompts, **greedy)

  total_data_ranking2chosen = []

  for prompt, gold, beam_out, greedy_out in tqdm(zip(prompts, golds, beam_outputs, greedy_outputs), total=len(prompts)):
    beam_result = extract_response(beam_out[0]['generated_text']) + tokenizer.eos_token
    greedy_result = extract_response(greedy_out[0]['generated_text']) + tokenizer.eos_token

    one_prompt_samples = [
        {'prompt': prompt, 'chosen': gold, 'rejected': beam_result},
        {'prompt': prompt, 'chosen': gold, 'rejected': greedy_result},
        {'prompt': prompt, 'chosen': beam_result, 'rejected': greedy_result},
    ]

    total_data_ranking2chosen.extend(one_prompt_samples)

  return total_data_ranking2chosen

In [10]:
def rm_dataset2_batched(dataset,generator=generator, tokenizer=tokenizer, greedy=greedy_generation_args, beam=beam_generation_args, batch_size=16):
  PROMPT_TEMPLATE = "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
  total_data_ranking2chosen = []

  prompts = []
  golds = []

  for tmp in dataset:
    prompt_text = PROMPT_TEMPLATE.format(prompt=tmp['prompt'])
    prompts.append(prompt_text)
    golds.append(tmp['response'] + tokenizer.eos_token)

  num_batches = (len(prompts) + batch_size - 1) // batch_size

  for batch_idx in tqdm(range(num_batches), desc="Generating in batches"):
    batch_prompts = prompts[batch_idx * batch_size : (batch_idx + 1) * batch_size]
    batch_golds   = golds[batch_idx * batch_size : (batch_idx + 1) * batch_size]

    beam_outputs = generator(batch_prompts, **beam)
    greedy_outputs = generator(batch_prompts, **greedy)

    for prompt, gold, beam_out, greedy_out in zip(batch_prompts, batch_golds, beam_outputs, greedy_outputs):
      beam_result = extract_response(beam_out[0]['generated_text']) + tokenizer.eos_token
      greedy_result = extract_response(greedy_out[0]['generated_text']) + tokenizer.eos_token

      one_prompt_samples = [
          {'prompt': prompt, 'chosen': gold, 'rejected': beam_result},
          {'prompt': prompt, 'chosen': gold, 'rejected': greedy_result},
          {'prompt': prompt, 'chosen': beam_result, 'rejected': greedy_result},
      ]
      total_data_ranking2chosen.extend(one_prompt_samples)

    del batch_prompts, batch_golds, beam_outputs, greedy_outputs
    gc.collect()
    torch.cuda.empty_cache()

  return total_data_ranking2chosen

In [11]:
torch.cuda.empty_cache()

In [11]:
from transformers import logging
logging.set_verbosity_error()

In [12]:
rm_train_dataset2 = rm_dataset2_batched(rm_train_dataset)
rm_val_dataset2 = rm_dataset2_batched(rm_val_dataset)

Generating in batches:   0%|          | 0/280 [00:00<?, ?it/s]

Generating in batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [15]:
rm_train_dataset3 = datasets.Dataset.from_list(rm_train_dataset2)
rm_val_dataset3 = datasets.Dataset.from_list(rm_val_dataset2)

In [18]:
rm_train_dataset3.save_to_disk('/content/drive/MyDrive/rm_train_dataset2')
rm_val_dataset3.save_to_disk('/content/drive/MyDrive/rm_val_dataset2')

Saving the dataset (0/1 shards):   0%|          | 0/13395 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1491 [00:00<?, ? examples/s]

In [3]:
rm_train_dataset3 = datasets.load_from_disk('/content/drive/MyDrive/rm_train_dataset2')
rm_val_dataset3 = datasets.load_from_disk('/content/drive/MyDrive/rm_val_dataset2')

In [4]:
from trl import RewardTrainer, RewardConfig
import torch.nn as nn

In [5]:
class RewardDataset(Dataset):
  def __init__(self, data, tokenizer, max_length=1024):
    self.data = data
    self.tokenizer = tokenizer
    self.max_length = max_length

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    prompt = self.data[idx]["prompt"]
    chosen = self.data[idx]["chosen"]
    rejected = self.data[idx]["rejected"]

    prompt_chosen = prompt + chosen
    prompt_rejected = prompt + rejected

    tokenized_chosen = self.tokenizer(prompt_chosen, padding = 'longest', truncation=True, max_length=self.max_length, return_tensors="pt")
    tokenized_rejected = self.tokenizer(prompt_rejected, padding = 'longest', truncation=True, max_length=self.max_length, return_tensors="pt")

    return {
        "input_ids_chosen": tokenized_chosen["input_ids"].squeeze(0),
        #"attention_mask_chosen": tokenized_chosen["attention_mask"],
        "input_ids_rejected": tokenized_rejected["input_ids"].squeeze(0),
        #"attention_mask_rejected": tokenized_rejected["attention_mask"],
    }

In [6]:
@dataclass
class RewardDataCollator:
    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, features):
        chosen_inputs = [f["input_ids_chosen"] for f in features]
        rejected_inputs = [f["input_ids_rejected"] for f in features]
        # try:
        #     batch_chosen = self.tokenizer.pad(
        #         {"input_ids": [x.tolist() for x in chosen_inputs]}, return_tensors="pt", padding=True
        #     )
        # except Exception as e:
        #   print("ERROR BATCH:")
        #   for i, x in enumerate(chosen_inputs):
        #     print(f"{i}: {x.shape}")
        #   raise e
        batch_chosen = self.tokenizer.pad(
            {"input_ids": chosen_inputs}, return_tensors="pt", padding=True
        )
        batch_rejected = self.tokenizer.pad(
            {"input_ids": rejected_inputs}, return_tensors="pt", padding=True
        )
        return {
            "input_ids_chosen": batch_chosen["input_ids"],
            "attention_mask_chosen": batch_chosen["attention_mask"],
            "input_ids_rejected": batch_rejected["input_ids"],
            "attention_mask_rejected": batch_rejected["attention_mask"],
        }

In [7]:
tokenizer = AutoTokenizer.from_pretrained("skt/ko-gpt-trinity-1.2B-v0.5")

rm_train_dataset4 = RewardDataset(
    rm_train_dataset3,
    tokenizer=tokenizer,
    max_length=1024
)

rm_val_dataset4 = RewardDataset(
    rm_val_dataset3,
    tokenizer=tokenizer,
    max_length=1024
)

rm_train_dataset4 = datasets.Dataset.from_list(rm_train_dataset4)
rm_val_dataset4 = datasets.Dataset.from_list(rm_val_dataset4)

rm_train_dataset4.set_format(type="torch")
rm_val_dataset4.set_format(type="torch")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from torch.nn.utils.rnn import pad_sequence
from transformers import DataCollatorWithPadding

input_ids_chosen = pad_sequence(chosen_inputs, batch_first=True, padding_value=tokenizer.pad_token_id)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [39]:
features = [
    {
        "input_ids_chosen": tokenizer("The cat sat on the mat.")["input_ids"],
        "input_ids_rejected": tokenizer("The cat is sitting.")["input_ids"],
    },
    {
        "input_ids_chosen": tokenizer("A quick brown fox.")["input_ids"],
        "input_ids_rejected": tokenizer("A slow yellow dog.")["input_ids"],
    },
]

for f in features:
    f["input_ids_chosen"] = torch.tensor(f["input_ids_chosen"])
    f["input_ids_rejected"] = torch.tensor(f["input_ids_rejected"])

collator = RewardDataCollator(tokenizer)
batch = collator([rm_train_dataset4[i] for i in range(1, 4)])

print(batch["input_ids_chosen"].shape)      # (batch_size, padded_seq_len_chosen)
print(batch["input_ids_rejected"].shape)    # (batch_size, padded_seq_len_rejected)
print(batch["attention_mask_rejected"])

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


torch.Size([3, 74])
torch.Size([3, 141])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   

In [9]:
data_collator=RewardDataCollator(tokenizer=tokenizer)

In [10]:
class GPTRewardModel(nn.Module):
  def __init__(self, model_path):
    super().__init__()
    self.backbone = AutoModel.from_pretrained(model_path)
    self.v_head = nn.Linear(self.backbone.config.hidden_size, 1)

  def forward(self, input_ids, attention_mask, **kwargs):
    out = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
    hidden = out.last_hidden_state
    # print("input_ids.shape =", input_ids.shape)
    # print("attention_mask.shape =", attention_mask.shape)
    # print(hidden.shape)
    last_token_index = attention_mask.sum(dim=1) - 1
    batch_idx = torch.arange(input_ids.size(0), device=input_ids.device)
    pooled = hidden[batch_idx, last_token_index]
    # print(pooled.shape)
    logits = self.v_head(pooled)
    # print(logits.shape)
    return logits.view(-1)

  @property
  def config(self):
    return self.backbone.config  # or self.backbone.model.config if needed

In [11]:
RM_model = GPTRewardModel(model_path="/content/drive/MyDrive/merged_model")

In [12]:
from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["c_attn", "c_proj"],
    bias="none",
    task_type=TaskType.SEQ_CLS
)

rm_model = get_peft_model(RM_model, lora_config)
rm_model.print_trainable_parameters()

trainable params: 4,055,040 || all params: 1,166,613,121 || trainable%: 0.3476


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [13]:
for name, param in rm_model.named_parameters():
    if "v_head" in name:
        param.requires_grad = True
        print(f"Trainable: {name}")

Trainable: base_model.model.v_head.weight
Trainable: base_model.model.v_head.bias


In [24]:
for name, param in rm_model.named_parameters():
    if param.requires_grad:
        print(f"{name}: {param.shape}")

base_model.model.backbone.h.0.attn.c_attn.lora_A.default.weight: torch.Size([8, 1920])
base_model.model.backbone.h.0.attn.c_attn.lora_B.default.weight: torch.Size([5760, 8])
base_model.model.backbone.h.0.attn.c_proj.lora_A.default.weight: torch.Size([8, 1920])
base_model.model.backbone.h.0.attn.c_proj.lora_B.default.weight: torch.Size([1920, 8])
base_model.model.backbone.h.0.mlp.c_proj.lora_A.default.weight: torch.Size([8, 7680])
base_model.model.backbone.h.0.mlp.c_proj.lora_B.default.weight: torch.Size([1920, 8])
base_model.model.backbone.h.1.attn.c_attn.lora_A.default.weight: torch.Size([8, 1920])
base_model.model.backbone.h.1.attn.c_attn.lora_B.default.weight: torch.Size([5760, 8])
base_model.model.backbone.h.1.attn.c_proj.lora_A.default.weight: torch.Size([8, 1920])
base_model.model.backbone.h.1.attn.c_proj.lora_B.default.weight: torch.Size([1920, 8])
base_model.model.backbone.h.1.mlp.c_proj.lora_A.default.weight: torch.Size([8, 7680])
base_model.model.backbone.h.1.mlp.c_proj.lora_

In [14]:
training_args = RewardConfig(
    output_dir="/content/drive/MyDrive/rm",
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=16,

    lr_scheduler_type="cosine",
    warmup_ratio=0.05,

    num_train_epochs=3,
    learning_rate=5e-4,

    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    greater_is_better=True,

    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,

    bf16=True,
    report_to="tensorboard",

    logging_strategy="steps",
    logging_dir="./logs",
    logging_steps=100,

    seed = 42,
    max_length = 1024,
    weight_decay = 0.01
)

In [15]:
if not hasattr(rm_model, "warnings_issued"):
  rm_model.warnings_issued = {}

In [66]:
class MyRewardTrainer(RewardTrainer):
  def compute_loss(
    self,
    model,
    inputs,
    return_outputs=False,
    num_items_in_batch=None
):
    print("[DEBUG] type(inputs):", type(inputs))
    print("[DEBUG] keys:", inputs.keys() if isinstance(inputs, dict) else "Not a dict")
    print("[DEBUG] input_ids_chosen shape:", inputs["input_ids_chosen"].shape if isinstance(inputs, dict) else "???")

    return super().compute_loss(model, inputs, return_outputs, num_items_in_batch)

In [16]:
class CustomRewardTrainer(RewardTrainer):
  def __init__(self, tokenizer=None, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.tokenizer = tokenizer

  def compute_loss(self, model, inputs, num_items_in_batch=None, return_outputs=False):
    """
    Compute pairwise loss: reward_chosen > reward_rejected.
    Loss = -log(sigmoid(reward_chosen - reward_rejected))
    """
    rewards_chosen = model(
        input_ids=inputs["input_ids_chosen"],
        attention_mask=inputs["attention_mask_chosen"]
    )

    rewards_rejected = model(
        input_ids=inputs["input_ids_rejected"],
        attention_mask=inputs["attention_mask_rejected"]
    )

    if rewards_chosen.ndim == 2:
      rewards_chosen = rewards_chosen[:, -1]
      rewards_rejected = rewards_rejected[:, -1]
    elif rewards_chosen.ndim == 1:
      pass
    else:
      raise ValueError(f"Unexpected reward shape: {rewards_chosen.shape}")

    loss = -torch.nn.functional.logsigmoid(rewards_chosen - rewards_rejected).mean()

    if return_outputs:
      return loss, {"chosen_reward": rewards_chosen, "rejected_reward": rewards_rejected}
    return loss

  def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
    model.eval()
    with torch.no_grad():
      reward_chosen = model(
          input_ids=inputs["input_ids_chosen"],
          attention_mask=inputs["attention_mask_chosen"]
      )

      reward_rejected = model(
          input_ids=inputs["input_ids_rejected"],
          attention_mask=inputs["attention_mask_rejected"]
      )

      reward_chosen = reward_chosen.view(-1)
      reward_rejected = reward_rejected.view(-1)

      rewards = torch.stack([reward_chosen, reward_rejected])  # [2, B]
      probs = rewards.softmax(dim=0).T  # [B, 2]

      labels = torch.zeros(probs.shape[0], dtype=torch.long, device=probs.device)

      if prediction_loss_only:
        return (None, None, None)
    return (None, probs, labels)

In [17]:
def compute_metrics(eval_preds):
  logits, labels = eval_preds
  preds = logits.argmax(axis=1)
  acc = (preds == labels).mean().item()
  return {"accuracy": acc}

In [18]:
trainer = CustomRewardTrainer(
    model=rm_model,
    args=training_args,
    train_dataset=rm_train_dataset4,
    eval_dataset=rm_val_dataset4,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    #peft_config=lora_config,
)

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
Trainer.tokenizer is now deprecated. You should use `Trainer.processing_class = processing_class` instead.


In [19]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
100,5.264100,No log,0.914152
200,3.009000,No log,0.927565
300,1.505600,No log,0.931590
400,1.460200,No log,0.937626
500,0.580700,No log,0.934943
600,0.321700,No log,0.934272


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                  ┃ rejected_text                                 ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ ### Instruction(명령어):                     │ ### Instruction(명령어):                      │ [0.9734, 0.0266] │
│ 다음 문장에 다섯 개의 동작 동사를            │ 다음 문장에 다섯 개의 동작 동사를 추가합니다. │                  │
│ 추가합니다.                                  │ 잭                                            │                  │
│ 잭                                           │                                               │                  │
│                                              │ ### Response(응답)::"잭은 노래하고 있습니다"… │                  │
│ ### Response(응답):Jack went running, Jack   │ "제이미는 팝콘을 먹습니다", "잭은 춤을 추고   │                  │
│ said hello, Jack jumped up and down, Jack    │ 있습니다".</s>                                │                  │
│ cried loudly, Jack danced happily</s>        │                                               │                  │
├──────────────────────────────────────────────┼───────────────────────────────────────────────┼──────────────────┤
│ ### Instruction(명령어):                     │ ### Instruction(명령어):                      │ [0.9909, 0.0091] │
│ 다음 문장에 다섯 개의 동작 동사를            │ 다음 문장에 다섯 개의 동작 동사를 추가합니다. │                  │
│ 추가합니다.                                  │ 잭                                            │                  │
│ 잭                                           │                                               │                  │
│                                              │ ### Response(응답):: 잭이 노래하는 것을       │                  │
│ ### Response(응답):Jack went running, Jack   │ 들었습니다.</s>                               │                  │
│ said hello, Jack jumped up and down, Jack    │                                               │                  │
│ cried loudly, Jack danced happily</s>        │                                               │                  │
├──────────────────────────────────────────────┼───────────────────────────────────────────────┼──────────────────┤
│ ### Instruction(명령어):                     │ ### Instruction(명령어):                      │ [0.7491, 0.2509] │
│ 다음 문장에 다섯 개의 동작 동사를            │ 다음 문장에 다섯 개의 동작 동사를 추가합니다. │                  │
│ 추가합니다.                                  │ 잭                                            │                  │
│ 잭                                           │                                               │                  │
│                                              │ ### Response(응답):: 잭이 노래하는 것을       │                  │
│ ### Response(응답)::"잭은 노래하고           │ 들었습니다.</s>                               │                  │
│ 있습니다", "제이미는 팝콘을 먹습니다", "잭 … │                                               │                  │
│ 춤을 추고 있습니다".</s>                     │                                               │                  │
├──────────────────────────────────────────────┼───────────────────────────────────────────────┼──────────────────┤
│ ### Instruction(명령어):                     │ ### Instruction(명령어):                      │ [0.9757, 0.0243] │
│ 일반적으로 토마토를 베이스로 한 소스와 함께  │ 일반적으로 토마토를 베이스로 한 소스와 함께   │                  │
│ 제공되는 파스타의 한 종류를 말하세요.        │ 제공되는 파스타의 한 종류를 말하세요.         │                  │
│                                              │                                               │                  │
│ ### Response(응답):"스파게티"</s>            │ ### Response(응답):토마토 소스                │                  │
│                                              │ 파스타입니다.</s>                             │                  │
└──────────────────────────────────────────────┴───────────────────────────────────────────────┴──────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                  ┃ rejected_text                                 ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ ### Instruction(명령어):                     │ ### Instruction(명령어):                      │ [0.9994, 0.0006] │
│ 다음 문장에 다섯 개의 동작 동사를            │ 다음 문장에 다섯 개의 동작 동사를 추가합니다. │                  │
│ 추가합니다.                                  │ 잭                                            │                  │
│ 잭                                           │                                               │                  │
│                                              │ ### Response(응답)::"잭은 노래하고 있습니다"… │                  │
│ ### Response(응답):Jack went running, Jack   │ "제이미는 팝콘을 먹습니다", "잭은 춤을 추고   │                  │
│ said hello, Jack jumped up and down, Jack    │ 있습니다".</s>                                │                  │
│ cried loudly, Jack danced happily</s>        │                                               │                  │
├──────────────────────────────────────────────┼───────────────────────────────────────────────┼──────────────────┤
│ ### Instruction(명령어):                     │ ### Instruction(명령어):                      │ [0.9999, 0.0001] │
│ 다음 문장에 다섯 개의 동작 동사를            │ 다음 문장에 다섯 개의 동작 동사를 추가합니다. │                  │
│ 추가합니다.                                  │ 잭                                            │                  │
│ 잭                                           │                                               │                  │
│                                              │ ### Response(응답):: 잭이 노래하는 것을       │                  │
│ ### Response(응답):Jack went running, Jack   │ 들었습니다.</s>                               │                  │
│ said hello, Jack jumped up and down, Jack    │                                               │                  │
│ cried loudly, Jack danced happily</s>        │                                               │                  │
├──────────────────────────────────────────────┼───────────────────────────────────────────────┼──────────────────┤
│ ### Instruction(명령어):                     │ ### Instruction(명령어):                      │ [0.9205, 0.0795] │
│ 다음 문장에 다섯 개의 동작 동사를            │ 다음 문장에 다섯 개의 동작 동사를 추가합니다. │                  │
│ 추가합니다.                                  │ 잭                                            │                  │
│ 잭                                           │                                               │                  │
│                                              │ ### Response(응답):: 잭이 노래하는 것을       │                  │
│ ### Response(응답)::"잭은 노래하고           │ 들었습니다.</s>                               │                  │
│ 있습니다", "제이미는 팝콘을 먹습니다", "잭 … │                                               │                  │
│ 춤을 추고 있습니다".</s>                     │                                               │                  │
├──────────────────────────────────────────────┼───────────────────────────────────────────────┼──────────────────┤
│ ### Instruction(명령어):                     │ ### Instruction(명령어):                      │ [0.9546, 0.0454] │
│ 일반적으로 토마토를 베이스로 한 소스와 함께  │ 일반적으로 토마토를 베이스로 한 소스와 함께   │                  │
│ 제공되는 파스타의 한 종류를 말하세요.        │ 제공되는 파스타의 한 종류를 말하세요.         │                  │
│                                              │                                               │                  │
│ ### Response(응답):"스파게티"</s>            │ ### Response(응답):토마토 소스                │                  │
│                                              │ 파스타입니다.</s>                             │                  │
└──────────────────────────────────────────────┴───────────────────────────────────────────────┴──────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                  ┃ rejected_text                                 ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ ### Instruction(명령어):                     │ ### Instruction(명령어):                      │ [0.997, 0.003]   │
│ 다음 문장에 다섯 개의 동작 동사를            │ 다음 문장에 다섯 개의 동작 동사를 추가합니다. │                  │
│ 추가합니다.                                  │ 잭                                            │                  │
│ 잭                                           │                                               │                  │
│                                              │ ### Response(응답)::"잭은 노래하고 있습니다"… │                  │
│ ### Response(응답):Jack went running, Jack   │ "제이미는 팝콘을 먹습니다", "잭은 춤을 추고   │                  │
│ said hello, Jack jumped up and down, Jack    │ 있습니다".</s>                                │                  │
│ cried loudly, Jack danced happily</s>        │                                               │                  │
├──────────────────────────────────────────────┼───────────────────────────────────────────────┼──────────────────┤
│ ### Instruction(명령어):                     │ ### Instruction(명령어):                      │ [0.9994, 0.0006] │
│ 다음 문장에 다섯 개의 동작 동사를            │ 다음 문장에 다섯 개의 동작 동사를 추가합니다. │                  │
│ 추가합니다.                                  │ 잭                                            │                  │
│ 잭                                           │                                               │                  │
│                                              │ ### Response(응답):: 잭이 노래하는 것을       │                  │
│ ### Response(응답):Jack went running, Jack   │ 들었습니다.</s>                               │                  │
│ said hello, Jack jumped up and down, Jack    │                                               │                  │
│ cried loudly, Jack danced happily</s>        │                                               │                  │
├──────────────────────────────────────────────┼───────────────────────────────────────────────┼──────────────────┤
│ ### Instruction(명령어):                     │ ### Instruction(명령어):                      │ [0.8439, 0.1561] │
│ 다음 문장에 다섯 개의 동작 동사를            │ 다음 문장에 다섯 개의 동작 동사를 추가합니다. │                  │
│ 추가합니다.                                  │ 잭                                            │                  │
│ 잭                                           │                                               │                  │
│                                              │ ### Response(응답):: 잭이 노래하는 것을       │                  │
│ ### Response(응답)::"잭은 노래하고           │ 들었습니다.</s>                               │                  │
│ 있습니다", "제이미는 팝콘을 먹습니다", "잭 … │                                               │                  │
│ 춤을 추고 있습니다".</s>                     │                                               │                  │
├──────────────────────────────────────────────┼───────────────────────────────────────────────┼──────────────────┤
│ ### Instruction(명령어):                     │ ### Instruction(명령어):                      │ [0.9959, 0.0041] │
│ 일반적으로 토마토를 베이스로 한 소스와 함께  │ 일반적으로 토마토를 베이스로 한 소스와 함께   │                  │
│ 제공되는 파스타의 한 종류를 말하세요.        │ 제공되는 파스타의 한 종류를 말하세요.         │                  │
│                                              │                                               │                  │
│ ### Response(응답):"스파게티"</s>            │ ### Response(응답):토마토 소스                │                  │
│                                              │ 파스타입니다.</s>                             │                  │
└──────────────────────────────────────────────┴───────────────────────────────────────────────┴──────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                  ┃ rejected_text                                 ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ ### Instruction(명령어):                     │ ### Instruction(명령어):                      │ [0.9996, 0.0004] │
│ 다음 문장에 다섯 개의 동작 동사를            │ 다음 문장에 다섯 개의 동작 동사를 추가합니다. │                  │
│ 추가합니다.                                  │ 잭                                            │                  │
│ 잭                                           │                                               │                  │
│                                              │ ### Response(응답)::"잭은 노래하고 있습니다"… │                  │
│ ### Response(응답):Jack went running, Jack   │ "제이미는 팝콘을 먹습니다", "잭은 춤을 추고   │                  │
│ said hello, Jack jumped up and down, Jack    │ 있습니다".</s>                                │                  │
│ cried loudly, Jack danced happily</s>        │                                               │                  │
├──────────────────────────────────────────────┼───────────────────────────────────────────────┼──────────────────┤
│ ### Instruction(명령어):                     │ ### Instruction(명령어):                      │ [1.0, 0.0]       │
│ 다음 문장에 다섯 개의 동작 동사를            │ 다음 문장에 다섯 개의 동작 동사를 추가합니다. │                  │
│ 추가합니다.                                  │ 잭                                            │                  │
│ 잭                                           │                                               │                  │
│                                              │ ### Response(응답):: 잭이 노래하는 것을       │                  │
│ ### Response(응답):Jack went running, Jack   │ 들었습니다.</s>                               │                  │
│ said hello, Jack jumped up and down, Jack    │                                               │                  │
│ cried loudly, Jack danced happily</s>        │                                               │                  │
├──────────────────────────────────────────────┼───────────────────────────────────────────────┼──────────────────┤
│ ### Instruction(명령어):                     │ ### Instruction(명령어):                      │ [0.9826, 0.0174] │
│ 다음 문장에 다섯 개의 동작 동사를            │ 다음 문장에 다섯 개의 동작 동사를 추가합니다. │                  │
│ 추가합니다.                                  │ 잭                                            │                  │
│ 잭                                           │                                               │                  │
│                                              │ ### Response(응답):: 잭이 노래하는 것을       │                  │
│ ### Response(응답)::"잭은 노래하고           │ 들었습니다.</s>                               │                  │
│ 있습니다", "제이미는 팝콘을 먹습니다", "잭 … │                                               │                  │
│ 춤을 추고 있습니다".</s>                     │                                               │                  │
├──────────────────────────────────────────────┼───────────────────────────────────────────────┼──────────────────┤
│ ### Instruction(명령어):                     │ ### Instruction(명령어):                      │ [0.9995, 0.0005] │
│ 일반적으로 토마토를 베이스로 한 소스와 함께  │ 일반적으로 토마토를 베이스로 한 소스와 함께   │                  │
│ 제공되는 파스타의 한 종류를 말하세요.        │ 제공되는 파스타의 한 종류를 말하세요.         │                  │
│                                              │                                               │                  │
│ ### Response(응답):"스파게티"</s>            │ ### Response(응답):토마토 소스                │                  │
│                                              │ 파스타입니다.</s>                             │                  │
└──────────────────────────────────────────────┴───────────────────────────────────────────────┴──────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ chosen_text                                      ┃ rejected_text                                   ┃ logits     ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ ### Instruction(명령어):                         │ ### Instruction(명령어):                        │ [1.0, 0.0] │
│ 다음 문장에 다섯 개의 동작 동사를 추가합니다.    │ 다음 문장에 다섯 개의 동작 동사를 추가합니다.   │            │
│ 잭                                               │ 잭                                              │            │
│                                                  │                                                 │            │
│ ### Response(응답):Jack went running, Jack said  │ ### Response(응답)::"잭은 노래하고 있습니다",   │            │
│ hello, Jack jumped up and down, Jack cried       │ "제이미는 팝콘을 먹습니다", "잭은 춤을 추고     │            │
│ loudly, Jack danced happily</s>                  │ 있습니다".</s>                                  │            │
├──────────────────────────────────────────────────┼─────────────────────────────────────────────────┼────────────┤
│ ### Instruction(명령어):                         │ ### Instruction(명령어):                        │ [1.0, 0.0] │
│ 다음 문장에 다섯 개의 동작 동사를 추가합니다.    │ 다음 문장에 다섯 개의 동작 동사를 추가합니다.   │            │
│ 잭                                               │ 잭                                              │            │
│                                                  │                                                 │            │
│ ### Response(응답):Jack went running, Jack said  │ ### Response(응답):: 잭이 노래하는 것을         │            │
│ hello, Jack jumped up and down, Jack cried       │ 들었습니다.</s>                                 │            │
│ loudly, Jack danced happily</s>                  │                                                 │            │
├──────────────────────────────────────────────────┼─────────────────────────────────────────────────┼────────────┤
│ ### Instruction(명령어):                         │ ### Instruction(명령어):                        │ [1.0, 0.0] │
│ 다음 문장에 다섯 개의 동작 동사를 추가합니다.    │ 다음 문장에 다섯 개의 동작 동사를 추가합니다.   │            │
│ 잭                                               │ 잭                                              │            │
│                                                  │                                                 │            │
│ ### Response(응답)::"잭은 노래하고 있습니다",    │ ### Response(응답):: 잭이 노래하는 것을         │            │
│ "제이미는 팝콘을 먹습니다", "잭은 춤을 추고      │ 들었습니다.</s>                                 │            │
│ 있습니다".</s>                                   │                                                 │            │
├──────────────────────────────────────────────────┼─────────────────────────────────────────────────┼────────────┤
│ ### Instruction(명령어):                         │ ### Instruction(명령어):                        │ [1.0, 0.0] │
│ 일반적으로 토마토를 베이스로 한 소스와 함께      │ 일반적으로 토마토를 베이스로 한 소스와 함께     │            │
│ 제공되는 파스타의 한 종류를 말하세요.            │ 제공되는 파스타의 한 종류를 말하세요.           │            │
│                                                  │                                                 │            │
│ ### Response(응답):"스파게티"</s>                │ ### Response(응답):토마토 소스                  │            │
│                                                  │ 파스타입니다.</s>                               │            │
└──────────────────────────────────────────────────┴─────────────────────────────────────────────────┴────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ chosen_text                                      ┃ rejected_text                                   ┃ logits     ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ ### Instruction(명령어):                         │ ### Instruction(명령어):                        │ [1.0, 0.0] │
│ 다음 문장에 다섯 개의 동작 동사를 추가합니다.    │ 다음 문장에 다섯 개의 동작 동사를 추가합니다.   │            │
│ 잭                                               │ 잭                                              │            │
│                                                  │                                                 │            │
│ ### Response(응답):Jack went running, Jack said  │ ### Response(응답)::"잭은 노래하고 있습니다",   │            │
│ hello, Jack jumped up and down, Jack cried       │ "제이미는 팝콘을 먹습니다", "잭은 춤을 추고     │            │
│ loudly, Jack danced happily</s>                  │ 있습니다".</s>                                  │            │
├──────────────────────────────────────────────────┼─────────────────────────────────────────────────┼────────────┤
│ ### Instruction(명령어):                         │ ### Instruction(명령어):                        │ [1.0, 0.0] │
│ 다음 문장에 다섯 개의 동작 동사를 추가합니다.    │ 다음 문장에 다섯 개의 동작 동사를 추가합니다.   │            │
│ 잭                                               │ 잭                                              │            │
│                                                  │                                                 │            │
│ ### Response(응답):Jack went running, Jack said  │ ### Response(응답):: 잭이 노래하는 것을         │            │
│ hello, Jack jumped up and down, Jack cried       │ 들었습니다.</s>                                 │            │
│ loudly, Jack danced happily</s>                  │                                                 │            │
├──────────────────────────────────────────────────┼─────────────────────────────────────────────────┼────────────┤
│ ### Instruction(명령어):                         │ ### Instruction(명령어):                        │ [1.0, 0.0] │
│ 다음 문장에 다섯 개의 동작 동사를 추가합니다.    │ 다음 문장에 다섯 개의 동작 동사를 추가합니다.   │            │
│ 잭                                               │ 잭                                              │            │
│                                                  │                                                 │            │
│ ### Response(응답)::"잭은 노래하고 있습니다",    │ ### Response(응답):: 잭이 노래하는 것을         │            │
│ "제이미는 팝콘을 먹습니다", "잭은 춤을 추고      │ 들었습니다.</s>                                 │            │
│ 있습니다".</s>                                   │                                                 │            │
├──────────────────────────────────────────────────┼─────────────────────────────────────────────────┼────────────┤
│ ### Instruction(명령어):                         │ ### Instruction(명령어):                        │ [1.0, 0.0] │
│ 일반적으로 토마토를 베이스로 한 소스와 함께      │ 일반적으로 토마토를 베이스로 한 소스와 함께     │            │
│ 제공되는 파스타의 한 종류를 말하세요.            │ 제공되는 파스타의 한 종류를 말하세요.           │            │
│                                                  │                                                 │            │
│ ### Response(응답):"스파게티"</s>                │ ### Response(응답):토마토 소스                  │            │
│                                                  │ 파스타입니다.</s>                               │            │
└──────────────────────────────────────────────────┴─────────────────────────────────────────────────┴────────────┘

TrainOutput(global_step=630, training_loss=1.9338672501700265, metrics={'train_runtime': 2221.3567, 'train_samples_per_second': 18.09, 'train_steps_per_second': 0.284, 'total_flos': 0.0, 'train_loss': 1.9338672501700265, 'epoch': 3.0})

In [21]:
model = rm_model.merge_and_unload()

In [22]:
print(model)

GPTRewardModel(
  (backbone): GPT2Model(
    (wte): Embedding(51200, 1920)
    (wpe): Embedding(1024, 1920)
    (drop): Dropout(p=0, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=5760, nx=1920)
          (c_proj): Conv1D(nf=1920, nx=1920)
          (attn_dropout): Dropout(p=0, inplace=False)
          (resid_dropout): Dropout(p=0, inplace=False)
        )
        (ln_2): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=7680, nx=1920)
          (c_proj): Conv1D(nf=1920, nx=7680)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
  )
  (v_head): Linear(in_features=1920, out_features=1, bias=True)
)


In [23]:
model.backbone.save_pretrained("/content/drive/MyDrive/rm_backbone")
torch.save(model.v_head.state_dict(), "/content/drive/MyDrive/rm_backbone/v_head.bin")

In [24]:
model_test = GPTRewardModel(model_path="/content/drive/MyDrive/rm_backbone")

In [25]:
v_head_weights = torch.load("/content/drive/MyDrive/rm_backbone/v_head.bin", map_location="cpu")
model_test.v_head.load_state_dict(v_head_weights)

<All keys matched successfully>